In [515]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [516]:
using SparseArrays, LinearAlgebra

In [517]:
function nodePositionVector(problem)
    dim = problem.dim
    non = problem.non
    if dim != 3
        error("nodePositionVector: This function only works for 3D problems.")
    end
    r = zeros(non * dim)
    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            nodeTags, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, -1, true, false)
            r[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            r[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            r[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
        end
    end
    return r
end

nodePositionVector (generic function with 1 method)

In [518]:
function stiffnessMatrixLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                #comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                #h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                #H = zeros(9 * numIntPoints, pdim * numNodes)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        #∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        #F1 = ∂H1 * r1
                        #F2 = reshape(F1, 3, 3)
                        #F2 = [1.0 0 0; 0 1.0 0; 0 0 1.0]
                        #display("F2=$F2")
                        #J1 = det(F2)
                        #display("J1=$J1")
                        #if J1 <= 0
                        #    display("J1=$J1")
                        #end
                        #C2 = F2' * F2
                        #iC2 = inv(C2)
                        #iC1 = reshape(iC2, 9, 1)
                        #iCiC1 = iC1[[1, 5, 9, 4, 6, 3]] * iC1[[1, 5, 9, 4, 6, 3]]'
                        #i1 = [1, 2, 3, 1, 2, 3]
                        #j1 = [1, 2, 3, 2, 3, 1]
                        #k1 = [1, 2, 3, 1, 2, 3]
                        #l1 = [1, 2, 3, 2, 3, 1]
                        #II1 = zeros(6, 6)
                        #for i2 in 1:6, j2 in 1:6
                        #    II1[i2, j2] = (iC2[i1[i2], k1[j2]] * iC2[j1[i2], l1[j2]] + iC2[i1[i2], l1[j2]] * iC2[j1[i2], k1[j2]]) / 2
                        #end
                        #C1 = λ * iCiC1 + 2 * (μ - λ * log(J1)) * II1
                        λ0 = λ
                        μ0 = μ
                        C1 = [λ0+2μ0 λ0 λ0 0 0 0;
                            λ0 λ0+2μ0 λ0 0 0 0;
                            λ0 λ0 λ0+2μ0 0 0 0;
                            0 0 0 μ0 0 0;
                            0 0 0 0 μ0 0;
                            0 0 0 0 0 μ0]
                        #display("C1 = $C1")
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        K1 += B1' * C1 * B1 * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixLinear (generic function with 1 method)

In [519]:
function stiffnessMatrixNonLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, dim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                for k in 1:numIntPoints, l in 1:numNodes
                    for kk in 1:9
                        H[k*9-(9-kk), l*9-(9-kk)] = h[(k-1)*numNodes+l]
                    end
                end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:dim
                        nn2[k:dim:dim*numNodes] = dim * nnet[j, 1:numNodes] .- (dim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    #S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        I3 = [1 0 0; 0 1 0; 0 0 1]
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        ∂H0 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        F1 = ∂H0 * r1
                        F2 = reshape(F1, 3, 3)
                        #if k == 0
                        #    display("F2=$F2")
                        #end
                        #J1 = det(F2)
                        E = (F2' * F2 - I3) / 2
                        #iC = inv(C)
                        S2 = 2μ * E + λ * (E[1, 1] + E[2, 2] + E[3, 3]) * I3
                        #if k == 0
                        #    display("S2=$S2")
                        #end
                        #S0 = reshape(H1 * S1, 3, 3)
                        ∂H1 = ∂H[k*9-(9-1):3:k*9-(9-9), 1:pdim*numNodes]
                        ∂H2 = ∂H[k*9-(9-2):3:k*9-(9-9), 1:pdim*numNodes]
                        ∂H3 = ∂H[k*9-(9-3):3:k*9-(9-9), 1:pdim*numNodes]
                        #display(∂H1)
                        #display(∂H2)
                        #display(∂H3)
                        #display("∂H1=$∂H1")
                        #display("∂H2=$∂H2")
                        #display("∂H3=$∂H3")
                        #∂H1 = ∂H[k*9-(9-1):k*9-(9-3), 1:pdim*numNodes]
                        #∂H2 = ∂H[k*9-(9-4):k*9-(9-6), 1:pdim*numNodes]
                        #∂H3 = ∂H[k*9-(9-7):k*9-(9-9), 1:pdim*numNodes]
                        #K1 += (∂H1' * S0 * ∂H1 + ∂H2' * S0 * ∂H2 + ∂H3' * S0 * ∂H3) * b * jacDet[k] * intWeights[k]
                        K1 += (∂H1' * S2 * ∂H1 + ∂H2' * S2 * ∂H2 + ∂H3' * S2 * ∂H3) * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixNonLinear (generic function with 1 method)

In [520]:
function stiffnessMatrixLinear0(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(4order + 1))
                numIntPoints = length(intWeights)
                #comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                #h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                #H = zeros(9 * numIntPoints, pdim * numNodes)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        F1 = ∂H1 * r1
                        F2 = reshape(F1, 3, 3)
                        J1 = det(F2)
                        C2 = F2' * F2
                        iC2 = inv(C2)
                        iC1 = reshape(C2, 9, 1)
                        iCiC1 = iC1[[1, 5, 9, 4, 6, 3]] * iC1[[1, 5, 9, 4, 6, 3]]'
                        i1 = [1, 2, 3, 1, 2, 3]
                        j1 = [1, 2, 3, 2, 3, 1]
                        k1 = [1, 2, 3, 1, 2, 3]
                        l1 = [1, 2, 3, 2, 3, 1]
                        II1 = zeros(6, 6)
                        for i2 in 1:6, j2 in 1:6
                            II1[i2, j2] = (iC2[i1[i2], k1[j2]] * iC2[j1[i2], l1[j2]] + iC2[i1[i2], l1[j2]] * iC2[j1[i2], k1[j2]]) / 2
                        end
                        C1 = λ * iCiC1 + 2 * (μ - λ * log(J1)) * II1
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        K1 += B1' * C1 * B1 * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixLinear0 (generic function with 1 method)

In [521]:
function loadVectorNonLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    #lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    #nn = []
    #I = []
    #J = []
    #V = []
    #V = convert(Vector{Float64}, V)
    #sizehint!(I, lengthOfIJV)
    #sizehint!(J, lengthOfIJV)
    #sizehint!(V, lengthOfIJV)
    f = zeros(problem.non * problem.pdim)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                f1 = zeros(pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    #display("nn2=$nn2")
                    #display("r1=$r1")
                    #S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    f1 .*= 0
                    for k in 1:numIntPoints
                        Ex = 10
                        νxy = 0.3
                        λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
                        μ = Ex / (2 * (1 + νxy))
                        I3 = [1 0 0; 0 1 0; 0 0 1]
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        ∂H0 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        F1 = ∂H0 * r1
                        F2 = reshape(F1, 3, 3)
                        #display("F2=$F2")
                        #J1 = det(F2)
                        E = (F2' * F2 - I3) / 2
                        #iC = inv(C)
                        S2 = 2μ * E + λ * (E[1, 1] + E[2, 2] + E[3, 3]) * I3
                        #S2 = [1 0 0; 0 0 0; 0 0 0]
                        #if k == 3 && j == 1 && false
                        #    display("S2=$S2")
                        #end
                        #S2 = -I3
                        #display("S2=$S2")
                        #S3 = reshape(S2, 9, 1)
                        #H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        #S0 = H1 * S1
                        #display("S: $(S2[[1, 5, 9, 4, 6, 3]])")
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        f1 += B1' * S2[[1, 5, 9, 4, 6, 3]] * b * jacDet[k] * intWeights[k]
                    end
                    #append!(I, nn2[Iidx[:]])
                    #append!(J, nn2[Jidx[:]])
                    #append!(V, K1[:])
                    f[nn2] += f1
                end
                #push!(nn, nnet)
            end
        end
    end
    #dof = problem.pdim * problem.non
    #K = sparse(I, J, V, dof, dof)
    #dropzeros!(K)
    return f
end

loadVectorNonLinear (generic function with 1 method)

In [522]:
function followerLoadVector(problem, r, loads)
    gmsh.model.setCurrent(problem.name)
    if !isa(loads, Vector)
        error("loadVector: loads are not arranged in a vector. Put them in [...]")
    end
    pdim = problem.pdim
    DIM = problem.dim
    b = problem.thickness
    non = problem.non
    dof = pdim * non
    fp = zeros(dof)
    ncoord2 = zeros(3 * problem.non)
    for n in 1:length(loads)
        name, fx, fy, fz = loads[n]
        if pdim == 3
            f = [.0, .0, .0]
        elseif pdim == 2
            f = [.0, .0]
        elseif pdim == 1
            f = [.0]
        else
            error("loadVector: dimension of the problem is $(problem.dim).")
        end
        dimTags = gmsh.model.getEntitiesForPhysicalName(name)
        for i ∈ 1:length(dimTags)
            dimTag = dimTags[i]
            dim = dimTag[1]
            tag = dimTag[2]
            elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)
            nodeTags::Vector{Int64}, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, tag, true, false)
            ncoord2[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
            for ii in 1:length(elementTypes)
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[ii])
                nnoe = reshape(elemNodeTags[ii], numNodes, :)'
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[ii], "Gauss" * string(order+1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[ii], intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[ii], intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elementTags[ii]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                H = zeros(pdim * numIntPoints, pdim * numNodes)
                for j in 1:numIntPoints
                    for k in 1:numNodes
                        for l in 1:pdim
                            H[j*pdim-(pdim-l), k*pdim-(pdim-l)] = h[k, j]
                        end
                    end
                end
                f1 = zeros(pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                ∂h = zeros(pdim, numNodes * numIntPoints)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                for l in 1:length(elementTags[ii])
                    elem = elementTags[ii][l]
                    for k in 1:numNodes
                        nnet[l, k] = elemNodeTags[ii][(l-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnoe[l, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    f1 .*= 0
                    ∂H .*= 0
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    display("r1=$r1")
                    for j in 1:numIntPoints
                        if DIM == 3 && pdim == 3
                            for l in 1:numNodes
                                ∂H[j*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (j-1)*numNodes+l]
                                ∂H[j*9-(9-2), l*pdim-(pdim-2)] = ∂h[1, (j-1)*numNodes+l]
                                ∂H[j*9-(9-3), l*pdim-(pdim-3)] = ∂h[1, (j-1)*numNodes+l]
                                ∂H[j*9-(9-4), l*pdim-(pdim-1)] = ∂h[2, (j-1)*numNodes+l]
                                ∂H[j*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (j-1)*numNodes+l]
                                ∂H[j*9-(9-6), l*pdim-(pdim-3)] = ∂h[2, (j-1)*numNodes+l]
                                ∂H[j*9-(9-7), l*pdim-(pdim-1)] = ∂h[3, (j-1)*numNodes+l]
                                ∂H[j*9-(9-8), l*pdim-(pdim-2)] = ∂h[3, (j-1)*numNodes+l]
                                ∂H[j*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (j-1)*numNodes+l]
                            end
                        else
                            error("nonlinearLoadVector: dim=$dim and pdim=$pdim is not yet implemented.")
                        end
                        ∂H1 = ∂H[j*9-(9-1):j*9, 1:pdim*numNodes]
                        F1 = ∂H1 * r1
                        display("F1=$F1")
                        F1 = reshape(F1, 3, 3)
                        display("F1=$F1")
                        x = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 2]
                        y = 0
                        z = 0
                        if isa(fx, Function) || isa(fy, Function) || isa(fz, Function)
                            y = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 1]
                            z = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 0]
                        end
                        if fz == 2im
                            if isa(fx, Function)
                                error("heatConvectionVector: h cannot be a function.")
                            end
                            f[1] = isa(fy, Function) ? fx * fy(x, y, z) : fx * fy
                        else
                            f[1] = isa(fx, Function) ? fx(x, y, z) : fx
                        end
                        if pdim > 1
                            f[2] = isa(fy, Function) ? fy(x, y, z) : fy
                        end
                        if pdim == 3
                            f[3] = isa(fz, Function) ? fz(x, y, z) : fz
                        end
                        r = x
                        H1 = H[j*pdim-(pdim-1):j*pdim, 1:pdim*numNodes] # H1[...] .= H[...] ????
                        ############### NANSON ######## 3D ###################################
                        if DIM == 3 && dim == 3
                            Ja = jacDet[j]
                        elseif DIM == 3 && dim == 2
                            xy = Jac[1, 3*j-2] * Jac[2, 3*j-1] - Jac[2, 3*j-2] * Jac[1, 3*j-1]
                            yz = Jac[2, 3*j-2] * Jac[3, 3*j-1] - Jac[3, 3*j-2] * Jac[2, 3*j-1]
                            zx = Jac[3, 3*j-2] * Jac[1, 3*j-1] - Jac[1, 3*j-2] * Jac[3, 3*j-1]
                            Ja = √(xy^2 + yz^2 + zx^2)
                        elseif DIM == 3 && dim == 1
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2 + (Jac[3, 3*j-2])^2)
                        elseif DIM == 3 && dim == 0
                            Ja = 1
                        ############ 2D #######################################################
                        elseif DIM == 2 && dim == 2 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = jacDet[j] * b
                        elseif DIM == 2 && dim == 2 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * jacDet[j] * r
                        elseif DIM == 2 && dim == 1 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * b
                        elseif DIM == 2 && dim == 1 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * r
                        elseif DIM == 2 && dim == 0
                            Ja = 1
                        ############ 1D #######################################################
                        else
                            error("loadVector: dimension of the problem is $(problem.dim), dimension of load is $dim.")
                        end
                        f1 += H1' * F1 * f * Ja * intWeights[j]
                    end
                    fp[nn2] += f1
                end
            end
        end
    end
    return fp
end



followerLoadVector (generic function with 1 method)

In [523]:
function deformationGradient(problem, r)
    gmsh.model.setCurrent(problem.name)

    type = :F
    nsteps = size(r, 2)
    F = []
    numElem = Int[]
    ncoord2 = zeros(3 * problem.non)
    dim = problem.dim
    pdim = problem.pdim
    non = problem.non

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim != 3 || problem.type != :Solid
            error("deformationGradient: dimension is $(problem.dim), problem type is $(problem.type). (They must be 3 and :Solid.)")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                nodeCoord = zeros(numNodes * 3)
                for k in 1:dim, j = 1:numNodes
                    nodeCoord[k+(j-1)*3] = localNodeCoord[k+(j-1)*dim]
                end
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, nodeCoord, "GradLagrange")
                ∇h = reshape(dfun, :, numNodes)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numNodes)
                ∂h = zeros(dim, numNodes * numNodes)
                ∂H = spzeros(9 * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, nodeCoord)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numNodes
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numNodes, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    ∂H .*= 0
                    for k in 1:numNodes
                        for l in 1:numNodes
                            ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                            ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                            ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                            ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                            ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                        end
                    end
                    push!(numElem, elem)
                    F1 = zeros(9numNodes, nsteps)
                    for k in 1:numNodes
                        ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                        for kk in 1:nsteps
                            F0 = ∂H1 * r[nn2, kk]
                            F1[(k-1)*9+1:k*9, kk] = [F0[1], F0[4], F0[7],
                                F0[2], F0[5], F0[8],
                                F0[3], F0[6], F0[9]]
                        end
                    end
                    push!(F, F1)
                end
            end
        end
    end
    sigma = FEM.TensorField(F, numElem, nsteps, type)
    return sigma
end

deformationGradient (generic function with 1 method)

In [524]:
import Base.*
function *(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("*(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("*(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            m = length(B.A[i]) ÷ 9
            if n != m
                error("*(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            end
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(reshape(A.A[i][9j-8:9j, k], 3, 3) * reshape(B.A[i][9j-8:9j, k], 3, 3), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

import Base.transpose
function transpose(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(transpose(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("transpose(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.adjoint
function adjoint(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(adjoint(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("adjoint(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

function unit(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape([1 0 0; 0 1 0; 0 0 1], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("unit(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

function traceI(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    trace = A.A[i][9j-8, k] + A.A[i][9j-4, k] + A.A[i][9j, k]
                    D[9j-8:9j, k] = reshape([trace 0 0; 0 trace 0; 0 0 trace], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("traceI(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

function detI(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    d = det(reshape(A.A[i][9j-8:9j, k], 3, 3))
                    D[9j-8:9j, k] = reshape([d 0 0; 0 d 0; 0 0 d], 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("detI(A::TensorField): TensorField type ($(A.type) is not yet implemented.")
    end
end

import Base.+
function +(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("+(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("*(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            m = length(B.A[i]) ÷ 9
            if n != m
                error("+(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            end
            D = A.A[i] + B.A[i]
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("+(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

import Base.-
function -(A::FEM.TensorField, B::FEM.TensorField)
    if (A.type == :s || A.type == :e || A.type == :F) && (B.type == :s || B.type == :e || B.type == :F)
        if length(A.A) != length(B.A)
            error("-(A::TensoeField, B::TensorField): size of A=$(length(A.A)) != size of B=$(length(B.A))")
        end
        nsteps = A.nsteps
        nsteps2 = B.nsteps
        if nsteps != nsteps2
            error("*(A::TensoeField, B::TensorField): nsteps of A=$(A.nsteps) != nsteps of B=$(B.nsteps)")
        end
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            m = length(B.A[i]) ÷ 9
            if n != m
                error("-(A::TensoeField, B::TensorField): size of A.A[$i]=$(9n) != size of B.A[$j]=$(9m)")
            end
            D = A.A[i] - B.A[i]
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("-(A::TensorField, B::TensorField): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function *(A::FEM.TensorField, b)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] * b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

function *(b, A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] * b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("*(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

import Base./
function /(A::FEM.TensorField, b)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            D = A.A[i] / b
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("/(A::TensorField, b): TensorField type ($(A.type) or $(B.type)) is not yet implemented.")
    end
end

import Base.inv
function inv(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(inv(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("inv(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.sqrt
function sqrt(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(sqrt(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("sqrt(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end

import Base.log
function log(A::FEM.TensorField)
    if A.type == :s || A.type == :e || A.type == :F
        nsteps = A.nsteps
        C = []
        for i in 1:length(A.A)
            n = length(A.A[i]) ÷ 9
            D = zeros(9n, nsteps)
            for j in 1:n
                for k in 1:nsteps
                    D[9j-8:9j, k] = reshape(log(reshape(A.A[i][9j-8:9j, k], 3, 3)), 9, 1)
                end
            end
            push!(C, D)
        end
        return FEM.TensorField(C, A.numElem, A.nsteps, :e)
    else
        error("log(A::TensorField): TensorField type ($(A.type)) is not yet implemented.")
    end
end


log (generic function with 27 methods)

In [525]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...                                                                                                                       
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.0010128s, CPU 0.001013s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 9

In [526]:
λ = 1
μ = 2

2

In [527]:
#mat = FEM.material("body", E=μ * (3λ + 2μ) / (λ + μ), ν=λ / 2 / (λ + μ))
mat = FEM.material("body", E=10, ν=0.3)
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


LowLevelFEM.Problem("body2", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 10.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 729)

In [528]:
fx(x, y, z) = 1000 * (y - 0.5)
supp1 = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
supp2 = FEM.displacementConstraint("bottom", uy=0)
supp3 = FEM.displacementConstraint("front", uz=0)
traction = FEM.load("right", fx=10)
traction2 = FEM.load("top", fy=-1)
traction3 = FEM.load("rear", fz=1)
bodyforce = FEM.load("body", fx=1)

q = FEM.solveDisplacement(problem, [traction, traction2, traction3], [supp1, supp2, supp3])
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [529]:
r0 = nodePositionVector(problem)
xx(x, y, z) = 1.1x
yy(x, y, z) = y
zz(x, y, z) = z
fld0 = FEM.field("body", fx=xx, fy=yy, fz=zz)
r1 = FEM.vectorField(problem, [fld0])
FEM.showDoFResults(problem, r1 - r0, :vector)

0

In [530]:
fnl0 = loadVectorNonLinear(problem, r0)
FEM.showDoFResults(problem, fnl0, :vector)

1

In [531]:
fnl1 = loadVectorNonLinear(problem, r1)
FEM.showDoFResults(problem, fnl1, :vector)

2

In [532]:
#fnl0 = followerLoadVector(problem, r, [traction])

In [533]:
Frigth = FEM.load("right", fx=1)
f0 = FEM.loadVector(problem, [Frigth])

2187-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [534]:
FEM.showDoFResults(problem, f0 - fnl1, :vector)

3

In [535]:
stiffnessMatrixNonLinear(problem, r1)

2187×2187 SparseMatrixCSC{Float64, Int64} with 107811 stored entries:
⎡⣿⣿⡟⢚⣿⣿⣿⣿⠓⠘⢻⡿⠚⣿⡕⢚⣗⣨⣿⠿⣽⣿⣿⢒⡀⣭⢺⠒⠀⠀⡄⢠⠗⠳⢨⡅⣷⠿⢬⡗⎤
⎢⣻⢉⣿⣿⣿⣏⠁⢨⣿⠿⢿⣃⣽⢻⢰⣾⣯⣭⣿⠛⠲⣿⢿⣭⢡⡍⣾⡆⠛⠀⠀⠶⠆⠛⠒⠰⠶⠞⠒⠶⎥
⎢⣿⣿⡿⢿⣿⣿⣯⣾⠟⠈⢿⣭⠉⣿⡷⢾⣯⣼⢿⣉⣶⣿⣷⢩⡄⣿⢺⣧⢠⡄⣤⢠⣭⢩⣴⡶⣶⣏⣰⡶⎥
⎢⣿⣿⡁⣀⣫⣿⢿⣷⣀⡀⢸⣿⣀⡿⡇⢈⣉⣉⣱⣦⣤⣻⣿⣘⢃⡛⣘⠿⣸⢇⣟⣸⡿⢸⢶⡆⡶⣦⣤⡄⎥
⎢⣙⠀⣿⡟⡛⠁⠀⠸⣿⣿⣽⡅⢿⣿⠘⣿⠛⠛⢛⣉⣉⢿⣯⠿⠸⠇⣿⡇⣿⢸⣿⣿⡇⣭⣭⢨⣭⣍⣉⡉⎥
⎢⣿⡶⠿⢳⡟⣷⣶⣶⠗⠿⢿⣷⣾⠟⠀⢻⣶⠀⢸⡿⠙⣿⣿⣖⠀⠀⠀⣶⣶⠈⠈⠉⣷⡿⠉⠈⠛⣿⠉⠛⎥
⎢⣾⣤⣷⣛⣧⣤⣤⡼⣿⣷⣾⠟⠿⣧⣤⣜⡟⢠⣼⠷⣤⣿⣧⠿⠀⣤⣤⡜⠿⠀⣤⣤⡝⠷⢠⣤⣤⡻⢠⣤⎥
⎢⣱⢉⣰⣶⣹⣏⡉⢉⣶⣤⣤⣀⣀⢿⢿⣷⣇⣸⣿⠀⢻⣿⣿⣀⣀⡹⣿⡇⠀⠀⠻⣾⡆⠀⠈⢿⣿⡄⠸⣿⎥
⎢⡙⣹⡏⣿⣋⣿⡇⢸⣿⠀⠘⠛⠛⣉⣉⣹⣿⣿⡍⠀⠀⣿⠸⢿⣿⣿⣭⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⡟⣿⠛⡟⢳⠱⣾⡟⢰⣶⡶⢶⡟⠛⠛⠃⠉⢻⣶⣖⠛⣖⠀⠈⠉⠛⠃⠀⠀⠀⠀⣶⣶⣶⣒⠒⣶⣶⡒⎥
⎢⣷⣿⣼⣦⣼⣿⣤⣻⣧⣜⣷⣤⣤⣿⣿⣶⣤⣤⣼⠙⠻⣦⣿⣤⣤⣤⣤⣤⣤⣤⣤⣤⣀⣉⣛⣻⣿⠟⠻⠿⎥
⎢⢻⢛⡟⣷⡝⣛⣛⢻⣯⡟⢻⢿⣭⡟⠛⢻⣶⣆⠘⠙⠛⣿⢻⣶⣿⣍⠉⣻⣿⣿⡛⠛⠋⠙⠛⠛⠛⠛⠛⠛⎥
⎢⡄⣬⡅⠶⣤⣭⣭⠰⠶⠆⠀⠀⠀⣤⣄⡸⣿⣿⡆⠀⠀⣿⡟⢿⣿⣿⣷⡝⢿⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢺⠒⠺⠿⠾⣶⣶⡜⠿⠿⢠⣤⣀⠿⠿⠿⠇⠻⠿⠀⠀⣿⣧⣠⣝⠿⠿⣧⣠⣝⠿⠿⣧⣠⡄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠛⠀⠀⠶⠶⢞⣛⣛⡘⠛⠛⠃⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⣿⣷⣄⢾⣿⣿⣷⣄⢿⣿⣿⣦⡀⠀⠀⠀⎥
⎢⠀⣉⢠⡄⠀⣛⣛⣹⣿⣿⡆⠀⠀⣿⣻⣦⠀⠀⠀⠀⠀⣿⣿⠈⠻⣿⣿⡇⠙⢿⣿⣿⡆⠙⢿⣿⣿⡄⠀⠀⎥
⎢⢽⡁⣬⠁⡇⣛⣛⣋⡍⣭⣽⡿⢷⡍⠈⠉⠀⠀⢸⣿⡄⢸⣏⠀⠀⠀⠉⣻⣿⣷⣌⠉⣿⣿⣷⣉⠉⣿⣧⠀⎥
⎢⠆⠶⢘⡀⢰⡿⠸⠷⡃⣛⡃⠀⠀⣶⣦⣄⠀⠀⢸⢻⣿⣸⣿⠀⠀⠀⠀⠉⠻⣿⣿⣷⡝⢻⣿⣿⣷⣻⣿⣧⎥
⎢⣽⡟⣸⠇⡼⢿⠸⣯⡇⢿⣿⣤⣤⡻⠛⠿⠀⠀⢸⣤⣿⠟⣿⠀⠀⠀⠀⠀⠀⠈⠛⠿⣧⣤⣽⣻⠿⣧⣽⡿⎥
⎣⢦⠷⢸⡄⢰⡾⠀⠿⡇⠸⣧⠀⠀⣶⣶⣦⠀⠀⢸⠻⣿⡆⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠛⠿⣿⣷⡿⢿⣷⎦

In [536]:
reshape(nodePositionVector(problem), 3, :)'

729×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0    0.0    1.0
 0.0    0.0    0.0
 0.0    1.0    1.0
 0.0    1.0    0.0
 1.0    0.0    1.0
 1.0    0.0    0.0
 1.0    1.0    1.0
 1.0    1.0    0.0
 0.0    0.0    0.25
 0.0    0.0    0.5
 ⋮             
 0.875  0.75   0.875
 0.875  0.625  0.875
 0.25   0.875  0.875
 0.125  0.875  0.875
 0.5    0.875  0.875
 0.375  0.875  0.875
 0.75   0.875  0.875
 0.625  0.875  0.875
 0.875  0.875  0.875

In [537]:
q = FEM.solveDisplacement(problem, [traction], [supp1])
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S)

u1 = FEM.showDoFResults(problem, q, :uvec, visible=false)
S1 = FEM.showDoFResults(problem, S, :s)

f = FEM.loadVector(problem, [traction])
r0 = nodePositionVector(problem)
n = 10
r = zeros(length(r0), n + 1)
q0 = zeros(length(r0), n)
r[:, 1] = r0

for i in 1:n
    F = deformationGradient(problem, r[:, i])

    Kl = stiffnessMatrixLinear(problem, r[:, i])
    Knl = stiffnessMatrixNonLinear(problem, r[:, i])
    f = FEM.loadVector(problem, [traction])
    fnl = loadVectorNonLinear(problem, r[:, i])
    #display("f=$f")
    #display("fnl=$fnl")
    K1, f1 = FEM.applyBoundaryConditions(problem, Kl + Knl, f - fnl, [supp1])
    display("f=$f")
    display("fnl=$fnl")
    display("f1=$f1")
    display("f-fnl=$(f-fnl)")
    q = FEM.solveDisplacement(K1, f1)
    #S = FEM.solveStress(problem, q)
    #S = FEM.elementsToNodes(problem, S)
    r[:, i+1] = r[:, i] + q
    display("$i: |q| = $(sum(abs, q))")
    display(reshape(q, 3, :)')
end

r0 = nodePositionVector(problem)
for i in 1:n+1
    r[:, i] -= r0
end
for i in 1:n
    q0[:, i] -= r[:, i+1] - r[:, i]
end
u2 = FEM.showDoFResults(problem, r, :uvec, t=1:n+1, visible=true)
q2 = FEM.showDoFResults(problem, q0, :uvec, t=1:n, visible=false, name="increment")
S2 = FEM.showDoFResults(problem, S, :s)

"f=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017361111111111122, 0.0, 0.0, 0.0173611111111111, 0.0, 0.0, 0.017361111111111067, 0.0, 0.0, 0.017361111111111115, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0" ⋯ 11676 bytes ⋯ "0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"

"fnl=[9.452142794586389e-18, -1.8400970975245816e-17, 2.401892758553085e-17, 4.692562459079341e-19, 8.311370516667546e-19, 2.9926738348733104e-19, -1.0053917235048599e-17, 4.454178285594733e-17, 4.313495145810034e-17, 4.9282475840658595e-18, -2.3934723493300145e-18, -1.0" ⋯ 51705 bytes ⋯ "8e-16, 1.758954635920614e-16, -9.246720686020638e-17, 7.709499163059319e-17, -2.9889440015586734e-16, -5.110749842280044e-16, 3.1515873791267874e-16, -8.222969744984272e-17, -7.966664989796174e-17, -1.229616664377602e-16, -2.189128356740458e-16, -2.323340697566672e-16]"

"f1=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017361111111111115, -1.579173346283055e-17, -1.7657298627311242e-17, 0.01736111111111112, 1.4168169887845224e-17, 6.7158985531978626e-18, 0.017361111111111015, 5.800020748083655e-17, -3.167291738271057e-1" ⋯ 46870 bytes ⋯ "743908e-16, -1.758954635920614e-16, 9.246720686020638e-17, -7.709499163059319e-17, 2.9889440015586734e-16, 5.110749842280044e-16, -3.1515873791267874e-16, 8.222969744984272e-17, 7.966664989796174e-17, 1.229616664377602e-16, 2.189128356740458e-16, 2.323340697566672e-16]"

"f-fnl=[-9.452142794586389e-18, 1.8400970975245816e-17, -2.401892758553085e-17, -4.692562459079341e-19, -8.311370516667546e-19, -2.9926738348733104e-19, 1.0053917235048599e-17, -4.454178285594733e-17, -4.313495145810034e-17, -4.9282475840658595e-18, 2.3934723493300145e-1" ⋯ 51441 bytes ⋯ "743908e-16, -1.758954635920614e-16, 9.246720686020638e-17, -7.709499163059319e-17, 2.9889440015586734e-16, 5.110749842280044e-16, -3.1515873791267874e-16, 8.222969744984272e-17, 7.966664989796174e-17, 1.229616664377602e-16, 2.189128356740458e-16, 2.323340697566672e-16]"

"1: |q| = 509.9728285354303"

...

"f1=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0001236223283724476, 1.6688581139851695e-5, 0.00028554718118601236, -4.923673339311818e-5, -5.191051494701322e-6, -1.8443456242503426e-5, 0.00012362232837230883, -1.668858113986701e-5, 0.00028554718118588" ⋯ 45731 bytes ⋯ "336867965, 0.00022020785438574443, -0.0023481857865579143, -0.0019223869452171805, 3.389227279467291e-5, -0.0018850181889734753, -0.0034225567156799626, 0.00013235088646339552, -0.0034169143236072205, -0.004035448890136899, 2.0423263924745425e-5, -0.003904019225347012]"

"f-fnl=[0.025586570522704733, 0.013585365318253285, -0.010850906473167278, 0.0261150961323174, 0.013416042460805081, 0.01554573114520726, 0.02558657052270492, -0.013585365318253434, -0.010850906473167391, 0.026115096132317262, -0.013416042460804987, 0.015545731145207214," ⋯ 49819 bytes ⋯ "336867965, 0.00022020785438574443, -0.0023481857865579143, -0.0019223869452171805, 3.389227279467291e-5, -0.0018850181889734753, -0.0034225567156799626, 0.00013235088646339552, -0.0034169143236072205, -0.004035448890136899, 2.0423263924745425e-5, -0.003904019225347012]"

"10: |q| = 0.3609556542432104"

729×3 adjoint(::Matrix{Float64}) with eltype Float64:
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.00273511    0.00040334    0.000151729
 -0.000743111  -0.000136429  -9.75279e-5
  0.00273511   -0.00040334    0.000151729
 -0.000743111   0.000136429  -9.75279e-5
  0.0           0.0           0.0
  0.0           0.0           0.0
  ⋮                          
 -0.00114355   -1.85059e-5    0.000108904
 -0.00117017   -1.08136e-5    8.52295e-5
 -0.000402201   4.74667e-5    3.93915e-5
 -0.000194602   2.49634e-5    3.8869e-5
 -0.00082961    6.53046e-5    4.16442e-5
 -0.000620848   6.40258e-5    4.31603e-5
 -0.00114015    1.5629e-5     3.70905e-5
 -0.00100685    4.64524e-5    3.86552e-5
 -0.00122431   -1.37279e-5    4.09925e-5

8

In [ ]:
f - fnl

In [ ]:
#gmsh.view.probe(u1, 1, 1, 0)

In [ ]:
#gmsh.view.probe(u2, 1, 1, 0)

In [ ]:
#gmsh.view.probe(S1, 1, 1, 0)

In [ ]:
#gmsh.view.probe(S2, 1, 1, 0)

In [ ]:
r = nodePositionVector(problem)
ux(x, y, z) = 0.5x
uy(x, y, z) = 0.5y
uz(x, y, z) = 0.5z
q0 = FEM.field("body", fx=ux, fy=uy, fz=uz)
q = FEM.vectorField(problem, [q0])
F = deformationGradient(problem, q)

In [ ]:
reshape(F.A[1][1:9], 3, 3)

In [ ]:
E = (F' * F - unit(F)) / 2

reshape(E.A[1][1:9], 3, 3)

In [ ]:
e = (unit(F) - inv(F * F')) / 2

reshape(e.A[1][1:9], 3, 3)

In [ ]:
e = inv(F') * E * inv(F)

reshape(e.A[1][1:9], 3, 3)

In [ ]:
U = sqrt(F' * F)

reshape(U.A[1][1:9], 3, 3)

In [ ]:
Ex = 10
νxy = 0.49999
λ = Ex * νxy / ((1 + νxy) * (1 - 2νxy))
μ = Ex / (2 * (1 + νxy))
I3 = unit(F)
iC = inv(F' * F)
J1 = detI(F)
SII = μ * (I3 - iC) + λ * log(J1) * inv(J1)

reshape(SII.A[1][1:9], 3, 3)

In [ ]:
σ = F * SII * F' * inv(detI(F))

reshape(σ.A[1][1:9], 3, 3)

In [ ]:
J = 0.5 * 0.5 * 0.5
μ / J * (J^(2 / 3) - 1) + λ / J * log(J)

In [538]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [514]:
gmsh.finalize()